In [3]:
import tensorflow as tf
import os
from tqdm import tqdm
import urllib.request as req

In [2]:
!pip install tqdm

You should consider upgrading via the 'C:\Users\pallavi.saxena\Documents\Mine\iNeuron\FullStackDataScienceNovember2021\FastTrack\3.DataScienceMasters\2.DL\1.FLive\Lecture16_NewCNNclassification\Codebase\CNN-classifier-FSDS\env\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 76.6/76.6 KB 1.4 MB/s eta 0:00:00
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)


In [4]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"

In [5]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file = "data.zip"

DESTINATION_ZIP_PATH = os.path.join(DESTINATION, data_file)

In [6]:
filename, headers = req.urlretrieve(SOURCE_URL, DESTINATION_ZIP_PATH)

In [7]:
filename

'data\\data.zip'

In [8]:
print(headers)

Content-Type: application/octet-stream
Accept-Ranges: bytes
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-blob-content-md5: l2WJyK3tt5/f2azyQ9eE8w==
Last-Modified: Sat, 13 Oct 2018 12:14:22 GMT
ETag: "0x8D6310568FD1DF6"
Content-Length: 824894548
Date: Sun, 03 Apr 2022 02:39:53 GMT
Connection: close




In [9]:
os.getcwd()

'C:\\Users\\pallavi.saxena\\Documents\\Mine\\iNeuron\\FullStackDataScienceNovember2021\\FastTrack\\3.DataScienceMasters\\2.DL\\1.FLive\\Lecture16_NewCNNclassification\\Codebase\\CNN-classifier-FSDS\\notebook'

In [10]:
from zipfile import ZipFile

with ZipFile(filename,"r") as zip_f:
    zip_f.extractall('data/')

In [11]:
TARGET_DATA_DIRS = ["Cat", "Dog"]
PARENT_DIR = os.path.join("data","PetImages")

In [14]:
from PIL import Image

In [18]:
import shutil
import imghdr

In [13]:
!pip install Pillow

     ---------------------------------------- 3.3/3.3 MB 7.4 MB/s eta 0:00:00

You should consider upgrading via the 'C:\Users\pallavi.saxena\Documents\Mine\iNeuron\FullStackDataScienceNovember2021\FastTrack\3.DataScienceMasters\2.DL\1.FLive\Lecture16_NewCNNclassification\Codebase\CNN-classifier-FSDS\env\python.exe -m pip install --upgrade pip' command.


In [16]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR, dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir, imgs)
        try:
            img = Image.open(path_to_img)
            img.verify()               
            #print(f"{path_to_img} is verified")
        except Exception as e:
            print(f"{path_to_img} is bad !")

data\PetImages\Cat\666.jpg is bad !
data\PetImages\Cat\Thumbs.db is bad !
data\PetImages\Dog\11702.jpg is bad !
data\PetImages\Dog\Thumbs.db is bad !


In [17]:
BAD_DATA_DIR = "bad_data"
os.makedirs(BAD_DATA_DIR, exist_ok=True)

In [19]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR, dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir, imgs)
        try:
            img = Image.open(path_to_img)
            img.verify()               
            #print(f"{path_to_img} is verified")
        except Exception as e:
            print(f"{path_to_img} is bad !")
            bad_data_path = os.path.join(BAD_DATA_DIR, imgs)
            shutil.move(path_to_img, bad_data_path)

data\PetImages\Cat\666.jpg is bad !
data\PetImages\Cat\Thumbs.db is bad !
data\PetImages\Dog\11702.jpg is bad !
data\PetImages\Dog\Thumbs.db is bad !


In [20]:
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

In [21]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 24998 files belonging to 2 classes.
Using 19999 files for training.
Found 24998 files belonging to 2 classes.
Using 4999 files for validation.


In [22]:
### Create a log dir for tensorboard logs

import time

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName = time.asctime().replace(" ", "_").replace(":", "")
    log_path = os.path.join(base_log_dir, uniqueName)
    print(f"saving logs at: {log_path}")
    return log_path

log_dir = get_log_path()

saving logs at: logs\fit\Sun_Apr__3_090652_2022


In [23]:
file_writer = tf.summary.create_file_writer(logdir=log_dir)

In [25]:
for images, labels in train_ds.take(2):
    print(images.shape, labels)

(32, 180, 180, 3) tf.Tensor([1 1 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0 0], shape=(32,), dtype=int32)
(32, 180, 180, 3) tf.Tensor([0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 1 1 0], shape=(32,), dtype=int32)


In [26]:
for images, labels in train_ds.take(1):
    print(images.shape, labels)

(32, 180, 180, 3) tf.Tensor([0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0], shape=(32,), dtype=int32)


In [27]:
images.shape

TensorShape([32, 180, 180, 3])

In [28]:
import numpy as np

In [30]:
with file_writer.as_default():
    images = np.reshape(images[10:30], (-1, 180, 180, 3))

    tf.summary.image("20 samples", images, max_outputs=25, step=0)

In [31]:
%load_ext tensorboard

In [33]:
%tensorboard --logdir=notebook/logs/fit/

https://www.tensorflow.org/tensorboard/get_started

In [37]:
### Create a log dir for tensorboard logs

import time
import numpy as np

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName = time.asctime().replace(" ", "_").replace(":", "")
    log_path = os.path.join(base_log_dir, uniqueName)
    print(f"saving logs at: {log_path}")
    return log_path

for images, labels in train_ds.take(1):
    print(images.shape, labels)
    
images.shape

(32, 180, 180, 3) tf.Tensor([0 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0], shape=(32,), dtype=int32)


TensorShape([32, 180, 180, 3])

In [38]:
log_dir = get_log_path()

file_writer = tf.summary.create_file_writer(logdir=log_dir)

with file_writer.as_default():
    images = np.array(images)       ##[10:30], (-1, 180, 180, 3))

    tf.summary.image("20 samples", images, max_outputs=25, step=0)

saving logs at: logs\fit\Sun_Apr__3_094207_2022


In [39]:
### Create a log dir for tensorboard logs

import time
import numpy as np

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName = time.asctime().replace(" ", "_").replace(":", "")
    log_path = os.path.join(base_log_dir, uniqueName)
    print(f"saving logs at: {log_path}")
    return log_path

for images, labels in train_ds.take(1):
    print(images.shape, labels)
    
images.shape

(32, 180, 180, 3) tf.Tensor([0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 1 1 0 1 1 0 1 0 0], shape=(32,), dtype=int32)


TensorShape([32, 180, 180, 3])

In [40]:
log_dir = get_log_path()

file_writer = tf.summary.create_file_writer(logdir=log_dir)

saving logs at: logs\fit\Sun_Apr__3_094706_2022


In [41]:
with file_writer.as_default():
    images = np.array(images) ##[10:30], (-1, 180, 180, 3))

    tf.summary.image("samples", images.astype("uint8"), max_outputs=25, step=0)